In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pyqpanda as pq
import pyvqnet as pv

In [2]:
class qm(pv.qnn.vqc.QModule):
    def __init__(self, name=""):
        super().__init__(name)
        self.device = pv.qnn.vqc.QMachine(2)
        self.RX1 = pv.qnn.vqc.qcircuit.RX(wires=0)
        self.RX2 = pv.qnn.vqc.qcircuit.RX(wires=1)
        self.RY1 = pv.qnn.vqc.qcircuit.RY(True, True, 0, wires=0)
        self.RY2 = pv.qnn.vqc.qcircuit.RY(True, True, 0, wires=1)
        self.RX3 = pv.qnn.vqc.qcircuit.RX(True, True, 0, wires=0)
        self.RX4 = pv.qnn.vqc.qcircuit.RX(True, True, 0, wires=1)
        self.CZ1 = pv.qnn.vqc.qcircuit.CZ(wires=(0, 1))
        self.CZ2 = pv.qnn.vqc.qcircuit.CZ(wires=(1, 0))
        self.RY3 = pv.qnn.vqc.qcircuit.RY(True, True, 0, wires=0)
        self.RY4 = pv.qnn.vqc.qcircuit.RY(True, True, 0, wires=1)
        self.measure = pv.qnn.vqc.MeasureAll(
            obs=[
                {
                    "wires": [0],
                    "observables": ["Z"],
                    "coefficient": [1],
                }
            ]
        )

    # @partial(pv.qnn.vqc.wrapper_compile)
    def forward(self, x, y):
        self.device.reset_states(x.shape[0])
        self.RX1(q_machine=self.device, params=x[:])
        self.RX2(q_machine=self.device, params=y[:])
        self.RY1(q_machine=self.device)
        self.RY2(q_machine=self.device)
        self.RX3(q_machine=self.device)
        self.RX4(q_machine=self.device)
        self.CZ1(q_machine=self.device)
        self.CZ2(q_machine=self.device)
        self.RY3(q_machine=self.device)
        self.RY4(q_machine=self.device)

        return self.measure(q_machine=self.device)


class QM(pv.nn.module.Module):
    def __init__(self, name=""):
        super().__init__(name)
        self.vqc = qm(name)

    def forward(self, x, y):
        X = pv.tensor.flatten(x, 1)
        Y = pv.tensor.flatten(y, 1)
        Z = pv.tensor.zeros_like(X)
        for i in range(X.shape[1]):
            Z[:, i] = self.vqc(X[:, i], Y[:, i])
        z = pv.tensor.reshape(Z, x.shape)
        return z


class Model(pv.nn.module.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = pv.nn.Conv2D(3, 4, (3, 3), (1, 1), "same")
        self.pool1 = pv.nn.MaxPool2D((4, 4), (4, 4))
        self.conv2 = pv.nn.Conv2D(3, 4, (3, 3), (1, 1), "same")
        self.pool2 = pv.nn.MaxPool2D((4, 4), (4, 4))
        self.vqc1 = QM("MyVQC1")
        self.conv3 = pv.nn.Conv2D(4, 8, (3, 3), (1, 1), "same")
        self.pool3 = pv.nn.MaxPool2D((4, 4), (4, 4))
        self.conv4 = pv.nn.Conv2D(4, 8, (3, 3), (1, 1), "same")
        self.pool4 = pv.nn.MaxPool2D((4, 4), (4, 4))
        self.vqc2 = QM("MyVQC2")
        self.fc = pv.nn.Linear(8 * 2 * 2, 10)

    def forward(self, x):
        x1 = 2 * pv.tensor.atan(self.pool1(self.conv1(x)))
        x2 = 2 * pv.tensor.atan(self.pool2(self.conv2(x)))
        x = self.vqc1(x1, x2)
        x1 = 2 * pv.tensor.atan(self.pool3(self.conv3(x)))
        x2 = 2 * pv.tensor.atan(self.pool4(self.conv4(x)))
        x = self.vqc2(x1, x2)
        x = pv.tensor.flatten(x, 1)
        x = self.fc(x)

        return x

In [3]:
epoch = 1000
batch = 32

In [4]:
m = Model()
print(pv.model_summary(m))
# print(m.parameters())
x = np.random.rand(batch, 3, 32, 32).astype("float32")
y_pred = m(x)
print(y_pred[0])
y = np.random.randint(0, 10, (batch,), dtype="int64")
print(y)
print(y_pred.argmax(1, False).to_numpy())



###################Model Summary#######################

classic layers: {'Conv2D': 4, 'MaxPool2D': 4, 'Linear': 1}
total classic parameters: 1146

qubits num: 2
gates: {'RX': 8, 'RY': 8, 'CZ': 4}
total quantum gates: 20
total quantum parameter gates: 16
total quantum parameters: 16
#########################################################
    
[-1.2253121,-0.0708358, 0.3842239,-0.5900414,-0.988984 , 0.1654306,
 -0.4705932, 1.0665956,-0.5896378,-0.0190593]
[7 9 7 3 9 8 5 8 1 6 9 6 8 4 8 4 2 6 7 1 8 2 8 1 4 9 3 8 7 7 7 3]
[7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]


In [5]:
los = pv.nn.loss.CrossEntropyLoss()
# opt = pv.optim.SGD(m.parameters())
opt = pv.optim.Adam(m.parameters())
for e in range(1, epoch + 1):
    opt.zero_grad()
    print(f"epoch {e} \t loss ", end="")
    y_pred = m(x)
    loss = los(y, y_pred)
    print(loss.item())
    y_p = y_pred.argmax(1, False).to_numpy()
    s = sum(y_p == y)
    print(y)
    print(y_p)
    print(f"correct: {s}")
    print()
    if s == y.shape[0]:
        print("done")
        break
    loss.backward()
    opt._step()

epoch 1 	 loss 2.3883109092712402
[7 9 7 3 9 8 5 8 1 6 9 6 8 4 8 4 2 6 7 1 8 2 8 1 4 9 3 8 7 7 7 3]
[7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]
correct: 6

epoch 2 	 loss 2.1956446170806885
[7 9 7 3 9 8 5 8 1 6 9 6 8 4 8 4 2 6 7 1 8 2 8 1 4 9 3 8 7 7 7 3]
[7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]
correct: 6

epoch 3 	 loss 2.129152297973633
[7 9 7 3 9 8 5 8 1 6 9 6 8 4 8 4 2 6 7 1 8 2 8 1 4 9 3 8 7 7 7 3]
[8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]
correct: 7

epoch 4 	 loss 2.1102633476257324
[7 9 7 3 9 8 5 8 1 6 9 6 8 4 8 4 2 6 7 1 8 2 8 1 4 9 3 8 7 7 7 3]
[8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]
correct: 7

epoch 5 	 loss 2.096712827682495
[7 9 7 3 9 8 5 8 1 6 9 6 8 4 8 4 2 6 7 1 8 2 8 1 4 9 3 8 7 7 7 3]
[8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]
correct: 7

epoch 6 	 loss 2.086745262145996
[7 9 7 3 9 8 5 8 1 6 9 6 8 4 8 4 2 6 7 1 8 2 8 1 4 9 3 8 7 7 7 3]
[8 8 8 8 8 8 

In [7]:
import time  
  
start_time = time.time()  
# 这里放你想要计时的代码  
time.sleep(2)  # 示例：模拟一段耗时的操作  
end_time = time.time()  
  
elapsed_time = end_time - start_time  
print(f"Elapsed time: {elapsed_time} seconds")

Elapsed time: 2.009093761444092 seconds
